# ĐỒ ÁN CUỐI KỲ - PHÂN TÍCH DỮ LIỆU BẢO TOÀN TÍNH RIÊNG TƯ

## THÀNH VIÊN NHÓM:

- 22120304 - Nguyễn Thị Kim Quý

- 22120338 - Đỗ Hạnh Thảo

- 22120352 - Phạm Nguyễn Quang Thoại

In [42]:
import pandas as pd
import numpy as np
import re

### 1. Đọc file

In [43]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print("Số dòng trong train.csv", len(train_data))
print(f"\nSố cột: {len(train_data.columns)}")
print("Tên các cột:", list(train_data.columns))
print("\n 5 dòng đầu trong train.csv:\n")
print(train_data.head())

Số dòng trong train.csv 1317

Số cột: 20
Tên các cột: ['Make', 'Model', 'Price', 'Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine', 'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity']

 5 dòng đầu trong train.csv:

        Make                                    Model    Price  Year  \
0  Ssangyong                               Rexton RX7   975000  2013   
1    Hyundai  Creta SX (O) 1.5 Petrol CVT [2020-2022]  1748999  2022   
2       Audi                     A4 2.0 TDI (143 bhp)  1150000  2012   
3    Hyundai        Grand i10 Magna AT 1.2 Kappa VTVT   549000  2018   
4    Hyundai                       Elite i20 Asta 1.2   675000  2017   

   Kilometer Fuel Type Transmission   Location   Color   Owner Seller Type  \
0      72000    Diesel    Automatic  Bangalore  Silver   First  Individual   
1       2670    Petrol    Automatic    Kolkata   White   First  Individual   
2   

In [ ]:
print("Số dòng trong test.csv", len(test_data))
print(f"\nSố cột: {len(test_data.columns)}")
print("Tên các cột:", list(test_data.columns))
print("\n 5 dòng đầu trong test.csv:\n")
print(test_data.head())

Số dòng trong test.csv 330

Số cột: 20
Tên các cột: ['Make', 'Model', 'Price', 'Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine', 'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity']

 5 dòng đầu trong test.csv:

            Make                           Model    Price  Year  Kilometer  \
0           Ford      Endeavour Trend 2.2 4x2 AT  2350000  2016      75000   
1         Toyota  Urban Cruiser Premium Grade AT  1050000  2021       1910   
2  Maruti Suzuki                    Alto 800 Lxi   210000  2014      42505   
3          Honda                  City SV Diesel   550000  2014      85000   
4        Hyundai                       Eon Era +   290000  2018      70000   

  Fuel Type Transmission  Location   Color   Owner Seller Type   Engine  \
0    Diesel    Automatic    Mohali   White  Second  Individual  2198 cc   
1    Petrol    Automatic  Varanasi   White   First  Ind

### 2. Tiền xử lý dữ liệu

In [45]:
# Hàm trích xuất số từ chuỗi
def extract_number(s):
    if pd.isna(s):
        return np.nan
    match = re.search(r'(\d+)', str(s).replace(',', ''))
    if match:
        return int(match.group(1))
    return np.nan

# Hàm label encode
# def label_encode(series, mapping=None):
#     if mapping is None:
#         uniques = series.dropna().unique()
#         mapping = {k: v for v, k in enumerate(sorted(uniques))}
#     encoded = series.map(mapping)
#     return encoded, mapping
def label_encode(series):
    uniques = series.dropna().unique()
    mapping = {k: v for v, k in enumerate(sorted(uniques))}
    encoded = series.map(mapping)
    return encoded, mapping

In [46]:
# Bước 1: Trích số từ các cột chuỗi số
for col in ['Engine', 'Max Power', 'Max Torque']:
    train_data[col] = train_data[col].apply(extract_number)
    test_data[col] = test_data[col].apply(extract_number)

# Bước 2: Label encoding thủ công cho các cột phân loại
categorical_cols = ['Make', 'Model', 'Fuel Type', 'Transmission', 'Location', 
                    'Color', 'Owner', 'Seller Type', 'Drivetrain']
mappings = {}

for col in categorical_cols:
    train_data[col], mappings[col] = label_encode(train_data[col])
    test_data[col] = test_data[col].map(mappings[col])

# Bước 3: Xử lý giá trị thiếu
# Với categorical NaN -> mean
for col in categorical_cols:
    mean = train_data[col].mean()
    train_data[col] = train_data[col].fillna(mean)
    test_data[col] = test_data[col].fillna(mean)
    
for col in categorical_cols:
    mean = train_data[col].mean()
    std = train_data[col].std()
    if std == 0:
        std = 1e-8  # tránh chia 0
    train_data[col] = (train_data[col] - mean) / std
    test_data[col] = (test_data[col] - mean) / std

# Với numeric NaN -> trung bình cột train
numeric_cols = ['Year', 'Kilometer', 'Engine', 'Max Power', 'Max Torque', 
                'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity']
for col in numeric_cols:
    mean = train_data[col].mean()
    train_data[col] = train_data[col].fillna(mean)
    test_data[col] = test_data[col].fillna(mean)

# Bước 4: Chuẩn hóa dữ liệu numeric
for col in numeric_cols:
    mean = train_data[col].mean()
    std = train_data[col].std()
    if std == 0:
        std = 1e-8  # tránh chia 0
    train_data[col] = (train_data[col] - mean) / std
    test_data[col] = (test_data[col] - mean) / std

# Bước 5: Tách X và Y
y_train = train_data['Price']
X_train = train_data.drop(columns=['Price'])
y_test = test_data['Price']
X_test = test_data.drop(columns=['Price'])

print("Sau khi tiển xử lý dữ liệu cho train_data: ")

print("Train.shape = ", X_train.shape)
print("Các cột trong X_train:", list(X_train.columns))
print("\n 5 dòng đầu trong X_train đã xử lý:\n")
print(X_train.head())

Sau khi tiển xử lý dữ liệu cho train_data: 
Train.shape =  (1317, 19)
Các cột trong X_train: ['Make', 'Model', 'Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine', 'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity']

 5 dòng đầu trong X_train đã xử lý:

       Make     Model      Year  Kilometer  Fuel Type  Transmission  Location  \
0  1.397994  0.572208 -0.989540   0.274506  -0.902065     -1.089406 -1.301665   
1 -0.778030 -0.749163  1.644419  -0.817523   1.088752     -1.089406  0.194324   
2 -1.624262 -1.573426 -1.282202  -0.229044  -0.902065     -1.089406  1.302465   
3 -0.778030 -0.026870  0.473770  -0.024767   1.088752     -1.089406 -0.304339   
4 -0.778030 -0.460246  0.181108  -0.094214   1.088752      0.917234 -0.304339   

      Color     Owner  Seller Type        Engine     Max Power    Max Torque  \
0  0.663426 -0.492233     0.186028  1.503194e+00  8.338177e-01  1.0

In [47]:
print("Sau khi tiền xử lý dữ liệu cho test_data:")
print("Test.shape = ", X_test.shape)
print("Các cột trong X_test:", list(X_test.columns))
print("\n 5 dòng đầu trong X_test đã xử lý:\n")
print(X_test.head())

Sau khi tiền xử lý dữ liệu cho test_data:
Test.shape =  (330, 19)
Các cột trong X_test: ['Make', 'Model', 'Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine', 'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity']

 5 dòng đầu trong X_test đã xử lý:

       Make     Model      Year  Kilometer  Fuel Type  Transmission  Location  \
0 -1.019811  0.000000 -0.111554   0.321759  -0.902065     -1.089406  0.637580   
1  1.639775  0.000000  1.351757  -0.829494   1.088752     -1.089406  2.133570   
2  0.430872 -1.475704 -0.696878  -0.190075   1.088752      0.917234  1.357872   
3 -0.898921 -0.982845 -0.696878   0.479271  -0.902065      0.917234 -0.193525   
4 -0.778030 -0.354026  0.473770   0.243003   1.088752      0.917234  0.138917   

      Color     Owner  Seller Type    Engine  Max Power  Max Torque  \
0  0.848421  1.636735     0.186028  0.744884   0.430370    0.974486   
1  0.848421

### 3.1 - 3.2: Xây dựng mô hình và huấn luyện

Mô hình: Linear Regression

Thuật toán tối ưu: Stochastic Gradient Descent

**Các cột dữ liệu được chọn**:

- Year: Năm sản xuất ảnh hưởng đến độ mới, giá trị còn lại.

- Kilometer: Số km đã đi là chỉ số hao mòn – ảnh hưởng trực tiếp đến giá.

- Fuel Type: Xe chạy xăng, dầu hay điện có mức giá khác nhau.

- Transmission: Xe số sàn thường rẻ hơn xe số tự động.

- Engine: Dung tích/mã lực động cơ ảnh hưởng đến giá xe.

- Max Power: Công suất phản ánh hiệu năng, thường đi kèm giá.

- Seating Capacity: Xe 4 chỗ, 7 chỗ, 9 chỗ có giá khác nhau.

- Fuel Tank Capacity: Phụ nhưng vẫn phản ánh phần nào kích thước/loại xe.

**Các công thức hồi quy sẽ sử dụng**:

- **y = a1x1 + a2x2 + a3x3 + a4x4**: Đây là mô hình hồi quy tuyến tính đơn giản, chuẩn, rất ổn cho baseline.

- **y = a1x1^2 + a2x2 + a3x3^2 + a4x4**: Đây là kỹ thuật polynomial features, giúp mô hình có thể nắm bắt được quan hệ phi tuyến.

- **y = a1(x1 + x2) + a3x3^ + a4x4**: Có các tương tác tiềm năng sau:

    - Year + Kilometer: Xe mới + số kilomet đã chạy ít -> giá cao

    - Make và Model: kết hợp thương hiệu + dòng xe đặc biệt -> đắt tiền.

    - Fuel Type và Engine / Max Power / Max Torque: Xe điện có thể có cấu hình động cơ và công suất khác biệt với xe xăng/dầu, tương tác giữa loại nhiên liệu và hiệu năng động cơ có thể quan trọng.

    - Seating Capacity và Fuel Tank Capacity: Xe nhiều chỗ thường có bình nhiên liệu lớn hơn, tương tác giúp phân biệt phân khúc xe.
    
    - Transmission và DriveTrain: Ví dụ xe số tự động + hệ dẫn động 4 bánh có thể giá cao hơn số sàn + dẫn động cầu trước.

- **y = a1x1x2 + a3x3^2**:
Cũng có các tương tác như trên

- **Regularized Linear Regression: Ridge Regression (L2 Regularization)**: λ∑(ai^2): Tránh overfitting

In [48]:
cols = ['Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Engine', 'Max Power', 'Seating Capacity', 'Fuel Tank Capacity']
X_train_official = X_train[cols]
X_test_official = X_test[cols]

print("Columns in X_train:", list(X_train_official.columns))
print("5 dòng dữ liệu đầu:\n", X_train_official.head())
print("Columns in X_test:", list(X_test_official.columns))
print("5 dòng dữ liệu đầu:\n", X_test_official.head())

Columns in X_train: ['Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Engine', 'Max Power', 'Seating Capacity', 'Fuel Tank Capacity']
5 dòng dữ liệu đầu:
        Year  Kilometer  Fuel Type  Transmission        Engine     Max Power  \
0 -0.989540   0.274506  -0.902065     -1.089406  1.503194e+00  8.338177e-01   
1  1.644419  -0.817523   1.088752     -1.089406 -3.225349e-01 -2.679054e-01   
2 -1.282202  -0.229044  -0.902065     -1.089406 -3.462240e-16  4.410261e-16   
3  0.473770  -0.024767   1.088752     -1.089406 -7.793477e-01 -7.644567e-01   
4  0.181108  -0.094214   1.088752      0.917234 -7.793477e-01 -7.489394e-01   

   Seating Capacity  Fuel Tank Capacity  
0      2.108120e+00        1.680819e+00  
1     -3.879255e-01       -1.589841e-01  
2     -1.108467e-15        4.668781e-16  
3     -3.879255e-01       -6.189349e-01  
4     -3.879255e-01       -4.875204e-01  
Columns in X_test: ['Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Engine', 'Max Power', 'Seating Capacity', 'Fu

In [60]:
# Tính mean và std của y_train
y_mean = y_train.mean()
y_std = y_train.std()

# Chuẩn hóa y_train
y_train_scaled = (y_train - y_mean) / y_std

In [57]:
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def sgd_linear_regression(X, y, transform_fn, lr=1e-6, max_epoch=500, lambda_l2=0.0):
    n_samples = X.shape[0]
    X_transformed = transform_fn(X)
    n_features = X_transformed.shape[1]
    weights = np.zeros(n_features)

    for epoch in range(max_epoch):
        indices = np.arange(n_samples)
        np.random.shuffle(indices)
        for i in indices:
            xi = X_transformed[i]
            yi = y[i]
            y_pred = np.dot(weights, xi)
            error = y_pred - yi
            gradient = error * xi + 2 * lambda_l2 * weights

            if np.any(np.isnan(gradient)) or np.any(np.isinf(gradient)):
                print(f"Warning: gradient NaN/Inf at epoch {epoch}, sample {i}")
                continue

            weights_new = weights - lr * gradient
            if np.any(np.isnan(weights_new)) or np.any(np.isinf(weights_new)):
                print(f"Warning: weights overflow at epoch {epoch}, sample {i}")
                continue

            weights = weights_new

        if epoch % 10 == 0:
            y_pred_scaled = np.dot(X_transformed, weights)
            
            # Tính MSE trên dữ liệu đã chuẩn hóa
            mse_scaled = mean_squared_error(y, y_pred_scaled)
            
            # Tính MSE trên đơn vị gốc
            y_pred_original = y_pred_scaled * y_std + y_mean
            y_original = y * y_std + y_mean
            mse_original = mean_squared_error(y_original, y_pred_original)
            
            print(f"Epoch {epoch}:")
            print(f"   ➤ MSE (chuẩn hóa): {mse_scaled:.4f}")
            print(f"   ➤ MSE (đơn vị gốc): {mse_original:.2f}")

    # Sau khi huấn luyện xong
    y_pred_scaled = np.dot(X_transformed, weights)
    y_pred_original = y_pred_scaled * y_std + y_mean
    y_original = y * y_std + y_mean
    mse_original = mean_squared_error(y_original, y_pred_original)

    return weights, mse_original


In [51]:
def formula_1(X):  
    return X[:, [0, 1, 2, 3, 4, 5, 6, 7]]

def formula_2(X):  
    return np.column_stack((X[:,0]**2, X[:,1], X[:,2]**2, X[:,3], X[:,4]**2, X[:,5], X[:,6]**2, X[:,7]))

def formula_3(X):  
    x1_plus_x2 = X[:,0] + X[:,1]
    x3_plus_x5 = X[:,2] + X[:,4]
    x7_plus_x8 = X[:,6] + X[:,7]
    return np.column_stack((x1_plus_x2, x3_plus_x5, X[:,3]**2, X[:,5], x7_plus_x8))

def formula_4(X):  
    x1x2 = X[:,0] * X[:,1]
    x3x5 = X[:,2] * X[:,4]
    x7x8 = X[:,6] * X[:,7]
    return np.column_stack((x1x2, x3x5, X[:,3]**2, X[:,5]**2, x7x8))

def formula_5_ridge(X):  
    return X

def run_all_formulas(X_train, y_train):
    formulas = [formula_1, formula_2, formula_3, formula_4, formula_5_ridge]
    names = [
        "Công thức 1: y = a1x1 + a2x2 + a3x3 + a4x4 + a5x5 + a6x6 + a7x7 + a8x8",
        "Công thức 2: y = a1x1^2 + a2x2 + a3x3^2 + a4x4 + a5x5^2 + a6x6 + a7x7^2 + a8x8",
        "Công thức 3: y = a1(x1 + x2) + a3(x3 + x5) + a4x4^2 + a7(x7 + x8)",
        "Công thức 4: y = a1x1x2 + a3x3x5 + a3x3^2 + a6x6^2 + a7x7x8",
        "Công thức 5: Ridge Regression (8 cột)"
    ]

    for i, formula in enumerate(formulas):
        lambda_l2 = 0.1 if i == 4 else 0.0
        weights, mse = sgd_linear_regression(X_train, y_train, formula, lambda_l2=lambda_l2)
        print(f"Công thức: {names[i]}")
        print(f"   ➤ MSE: {mse:.2f}")
        print(f"   ➤ Weights: {weights}\n")

In [52]:
# run_all_formulas(X_train_official.values, y_train_scaled.values)

In [58]:
weights1, mse1 = sgd_linear_regression(X_train_official.values, y_train_scaled.values, formula_1, lambda_l2=0.0)
print(f"Công thức 1:")
print(f"MSE: {mse1:.2f}")
print(f"Weights: {weights1}\n")

Epoch 0:
   ➤ MSE (chuẩn hóa): 0.9948
   ➤ MSE (đơn vị gốc): 5766812175923.53
Epoch 10:
   ➤ MSE (chuẩn hóa): 0.9528
   ➤ MSE (đơn vị gốc): 5523087344554.51
Epoch 20:
   ➤ MSE (chuẩn hóa): 0.9140
   ➤ MSE (đơn vị gốc): 5297972439735.58
Epoch 30:
   ➤ MSE (chuẩn hóa): 0.8781
   ➤ MSE (đơn vị gốc): 5089975945321.94
Epoch 40:
   ➤ MSE (chuẩn hóa): 0.8449
   ➤ MSE (đơn vị gốc): 4897740326725.27
Epoch 50:
   ➤ MSE (chuẩn hóa): 0.8143
   ➤ MSE (đơn vị gốc): 4720010280618.36
Epoch 60:
   ➤ MSE (chuẩn hóa): 0.7859
   ➤ MSE (đơn vị gốc): 4555632152727.39
Epoch 70:
   ➤ MSE (chuẩn hóa): 0.7597
   ➤ MSE (đơn vị gốc): 4403544981827.48
Epoch 80:
   ➤ MSE (chuẩn hóa): 0.7354
   ➤ MSE (đơn vị gốc): 4262776587794.74
Epoch 90:
   ➤ MSE (chuẩn hóa): 0.7129
   ➤ MSE (đơn vị gốc): 4132434110368.62
Epoch 100:
   ➤ MSE (chuẩn hóa): 0.6921
   ➤ MSE (đơn vị gốc): 4011688386275.52
Epoch 110:
   ➤ MSE (chuẩn hóa): 0.6728
   ➤ MSE (đơn vị gốc): 3899799730615.72
Epoch 120:
   ➤ MSE (chuẩn hóa): 0.6549
   ➤ MSE (đ

In [62]:
weights2, mse2 = sgd_linear_regression(X_train_official.values, y_train_scaled.values, formula_2, lambda_l2=0.0)
print(f"Công thức 2:")
print(f"MSE: {mse2:.2f}")
print(f"Weights: {weights2}\n")

Epoch 0:
   ➤ MSE (chuẩn hóa): 0.9907
   ➤ MSE (đơn vị gốc): 5742958357425.19
Epoch 10:
   ➤ MSE (chuẩn hóa): 0.9169
   ➤ MSE (đơn vị gốc): 5314780477633.46
Epoch 20:
   ➤ MSE (chuẩn hóa): 0.8594
   ➤ MSE (đơn vị gốc): 4981683335215.94
Epoch 30:
   ➤ MSE (chuẩn hóa): 0.8140
   ➤ MSE (đơn vị gốc): 4718404677197.65
Epoch 40:
   ➤ MSE (chuẩn hóa): 0.7775
   ➤ MSE (đơn vị gốc): 4506744588186.97
Epoch 50:
   ➤ MSE (chuẩn hóa): 0.7476
   ➤ MSE (đơn vị gốc): 4333419954902.30
Epoch 60:
   ➤ MSE (chuẩn hóa): 0.7226
   ➤ MSE (đơn vị gốc): 4188872247974.14
Epoch 70:
   ➤ MSE (chuẩn hóa): 0.7015
   ➤ MSE (đơn vị gốc): 4066170595453.59
Epoch 80:
   ➤ MSE (chuẩn hóa): 0.6832
   ➤ MSE (đơn vị gốc): 3960207078921.88
Epoch 90:
   ➤ MSE (chuẩn hóa): 0.6672
   ➤ MSE (đơn vị gốc): 3867254987491.13
Epoch 100:
   ➤ MSE (chuẩn hóa): 0.6529
   ➤ MSE (đơn vị gốc): 3784570122392.93
Epoch 110:
   ➤ MSE (chuẩn hóa): 0.6400
   ➤ MSE (đơn vị gốc): 3710135800209.95
Epoch 120:
   ➤ MSE (chuẩn hóa): 0.6284
   ➤ MSE (đ

In [63]:
weights3, mse3 = sgd_linear_regression(X_train_official.values, y_train_scaled.values, formula_3, lambda_l2=0.0)
print(f"Công thức 1:")
print(f"MSE: {mse3:.2f}")
print(f"Weights: {weights3}\n")

Epoch 0:
   ➤ MSE (chuẩn hóa): 0.9962
   ➤ MSE (đơn vị gốc): 5774455449919.46
Epoch 10:
   ➤ MSE (chuẩn hóa): 0.9667
   ➤ MSE (đơn vị gốc): 5603474866854.01
Epoch 20:
   ➤ MSE (chuẩn hóa): 0.9393
   ➤ MSE (đơn vị gốc): 5444791168760.74
Epoch 30:
   ➤ MSE (chuẩn hóa): 0.9139
   ➤ MSE (đơn vị gốc): 5297448934854.51
Epoch 40:
   ➤ MSE (chuẩn hóa): 0.8903
   ➤ MSE (đơn vị gốc): 5160542365962.80
Epoch 50:
   ➤ MSE (chuẩn hóa): 0.8683
   ➤ MSE (đơn vị gốc): 5033259463321.89
Epoch 60:
   ➤ MSE (chuẩn hóa): 0.8479
   ➤ MSE (đơn vị gốc): 4914840178423.11
Epoch 70:
   ➤ MSE (chuẩn hóa): 0.8289
   ➤ MSE (đơn vị gốc): 4804603955085.09
Epoch 80:
   ➤ MSE (chuẩn hóa): 0.8111
   ➤ MSE (đơn vị gốc): 4701915476481.89
Epoch 90:
   ➤ MSE (chuẩn hóa): 0.7946
   ➤ MSE (đơn vị gốc): 4606182263144.64
Epoch 100:
   ➤ MSE (chuẩn hóa): 0.7792
   ➤ MSE (đơn vị gốc): 4516868764556.15
Epoch 110:
   ➤ MSE (chuẩn hóa): 0.7648
   ➤ MSE (đơn vị gốc): 4433477998010.97
Epoch 120:
   ➤ MSE (chuẩn hóa): 0.7514
   ➤ MSE (đ

In [64]:
weights4, mse4 = sgd_linear_regression(X_train_official.values, y_train_scaled.values, formula_4, lambda_l2=0.0)
print(f"Công thức 1:")
print(f"MSE: {mse4:.2f}")
print(f"Weights: {weights4}\n")

Epoch 0:
   ➤ MSE (chuẩn hóa): 0.9912
   ➤ MSE (đơn vị gốc): 5745904535835.64
Epoch 10:
   ➤ MSE (chuẩn hóa): 0.9224
   ➤ MSE (đơn vị gốc): 5346994354760.88
Epoch 20:
   ➤ MSE (chuẩn hóa): 0.8704
   ➤ MSE (đơn vị gốc): 5045140403375.17
Epoch 30:
   ➤ MSE (chuẩn hóa): 0.8309
   ➤ MSE (đơn vị gốc): 4816468347608.39
Epoch 40:
   ➤ MSE (chuẩn hóa): 0.8010
   ➤ MSE (đơn vị gốc): 4643170875361.90
Epoch 50:
   ➤ MSE (chuẩn hóa): 0.7783
   ➤ MSE (đơn vị gốc): 4511690684416.90
Epoch 60:
   ➤ MSE (chuẩn hóa): 0.7611
   ➤ MSE (đơn vị gốc): 4411755170457.61
Epoch 70:
   ➤ MSE (chuẩn hóa): 0.7480
   ➤ MSE (đơn vị gốc): 4335712894526.76
Epoch 80:
   ➤ MSE (chuẩn hóa): 0.7380
   ➤ MSE (đơn vị gốc): 4277662613816.07
Epoch 90:
   ➤ MSE (chuẩn hóa): 0.7303
   ➤ MSE (đơn vị gốc): 4233262936541.43
Epoch 100:
   ➤ MSE (chuẩn hóa): 0.7244
   ➤ MSE (đơn vị gốc): 4199184640125.69
Epoch 110:
   ➤ MSE (chuẩn hóa): 0.7199
   ➤ MSE (đơn vị gốc): 4172870313335.71
Epoch 120:
   ➤ MSE (chuẩn hóa): 0.7163
   ➤ MSE (đ

In [65]:
weights5, mse5 = sgd_linear_regression(X_train_official.values, y_train_scaled.values, formula_5_ridge, lambda_l2=0.1)
print(f"Công thức 1:")
print(f"MSE: {mse5:.2f}")
print(f"Weights: {weights5}\n")

Epoch 0:
   ➤ MSE (chuẩn hóa): 0.9948
   ➤ MSE (đơn vị gốc): 5766814904533.90
Epoch 10:
   ➤ MSE (chuẩn hóa): 0.9529
   ➤ MSE (đơn vị gốc): 5523464136602.13
Epoch 20:
   ➤ MSE (chuẩn hóa): 0.9142
   ➤ MSE (đơn vị gốc): 5299257164920.65
Epoch 30:
   ➤ MSE (chuẩn hóa): 0.8785
   ➤ MSE (đơn vị gốc): 5092607184120.93
Epoch 40:
   ➤ MSE (chuẩn hóa): 0.8457
   ➤ MSE (đơn vị gốc): 4902040738513.99
Epoch 50:
   ➤ MSE (chuẩn hóa): 0.8153
   ➤ MSE (đơn vị gốc): 4726239157513.97
Epoch 60:
   ➤ MSE (chuẩn hóa): 0.7873
   ➤ MSE (đơn vị gốc): 4563983552072.64
Epoch 70:
   ➤ MSE (chuẩn hóa): 0.7615
   ➤ MSE (đơn vị gốc): 4414146467387.48
Epoch 80:
   ➤ MSE (chuẩn hóa): 0.7376
   ➤ MSE (đơn vị gốc): 4275715214688.06
Epoch 90:
   ➤ MSE (chuẩn hóa): 0.7155
   ➤ MSE (đơn vị gốc): 4147752412893.22
Epoch 100:
   ➤ MSE (chuẩn hóa): 0.6951
   ➤ MSE (đơn vị gốc): 4029402437352.97
Epoch 110:
   ➤ MSE (chuẩn hóa): 0.6762
   ➤ MSE (đơn vị gốc): 3919892874919.84
Epoch 120:
   ➤ MSE (chuẩn hóa): 0.6587
   ➤ MSE (đ

In [ ]:
mse_list = [mse1, mse2, mse3, mse4, mse5]
best_formula_index = mse_list.index(min(mse_list)) + 1  

print(f"Công thức tốt nhất là công thức {best_formula_index} với MSE = {min(mse_list):.2f}")

Công thức tốt nhất là công thức 1 với MSE = 2460082248656.69


In [68]:
# Tính mean và std của y_train
y_test_mean = y_train.mean()
y_test_std = y_train.std()

# Chuẩn hóa y_train
y_test_scaled = (y_test - y_test_mean) / y_test_std

In [70]:
mse_test_list = []
mse_test_scaled_list = []

for i, (weights, formula) in enumerate(zip(
    [weights1, weights2, weights3, weights4, weights5],
    [formula_1, formula_2, formula_3, formula_4, formula_5_ridge]
), 1):
    # Biến đổi X_test theo công thức
    X_test_transformed = formula(X_test_official.values)
    
    # Dự đoán chuẩn hóa
    y_pred_scaled = np.dot(X_test_transformed, weights)
    
    # Dự đoán đơn vị gốc
    y_pred_original = y_pred_scaled * y_std + y_mean

    # MSE chuẩn hóa (so với y_test_scaled)
    mse_scaled = mean_squared_error(y_test_scaled, y_pred_scaled)
    mse_test_scaled_list.append(mse_scaled)

    # MSE đơn vị gốc (so với y_test thật)
    mse_original = mean_squared_error(y_test, y_pred_original)
    mse_test_list.append(mse_original)

    # In kết quả
    print(f"Công thức {i}:")
    print(f"   ➤ MSE (chuẩn hóa): {mse_scaled:.4f}")
    print(f"   ➤ MSE (đơn vị gốc): {mse_original:.2f}")
    print("-" * 40)

# Tìm công thức tốt nhất (dựa trên MSE gốc)
best_index = mse_test_list.index(min(mse_test_list)) + 1
print(f"\n✅ Công thức tốt nhất trên tập test là: Công thức {best_index}")

Công thức 1:
   ➤ MSE (chuẩn hóa): 0.3195
   ➤ MSE (đơn vị gốc): 1852326016788.84
----------------------------------------
Công thức 2:
   ➤ MSE (chuẩn hóa): 0.3305
   ➤ MSE (đơn vị gốc): 1915802872403.58
----------------------------------------
Công thức 3:
   ➤ MSE (chuẩn hóa): 0.4124
   ➤ MSE (đơn vị gốc): 2390467716914.80
----------------------------------------
Công thức 4:
   ➤ MSE (chuẩn hóa): 0.4656
   ➤ MSE (đơn vị gốc): 2698685116300.72
----------------------------------------
Công thức 5:
   ➤ MSE (chuẩn hóa): 0.3264
   ➤ MSE (đơn vị gốc): 1892030826118.38
----------------------------------------

✅ Công thức tốt nhất trên tập test là: Công thức 1


### 3.3 Thêm nhiễu